# Ordinary Least Squares
## Course (very short) recap
This lab consists in implementing the **Ordinary Least Squares** (OLS) algorithm, which is **a linear regression with a least-squares penalty**. Given a training set $ D = \left\{ \left(x^{(i)}, y^{(i)}\right), x^{(i)} \in \mathcal{X}, y^{(i)} \in \mathcal{Y}, i \in \{1, \dots, n \}  \right\}$, recall (from lectures 1 and 2) OLS aims at minimizing the following cost function $J$:
$$J(\theta) = \dfrac{1}{2} \sum_{i = 1}^{n} \left( h\left(x^{(i)}\right) - y^{(i)} \right)^2$$
where 
$$h(x) = \sum_{j = 0}^{d} \theta_j x_j = \theta^T x.$$

For the sake of simplicity, we will be working on a small training set (the one we used in lectures 1 and 2) $D$:

| living area (m$^2$) | price (1000's BGN)|
|--------------------:|------------------:|
|                 50  |         30        |
|                 76  |         48        |
|                 26  |         12        |
|                102  |         90        |

## Defining the training set
**Exercise 1**: Define variables that will contain the training set.

**Note**: Do not forget the intercept!

In [1]:
X = [[50., 1.], [76., 1.], [26., 1.], [102., 1.]]
Y = [30., 48., 12., 90.]

In this very simple example, the dimensionality is $d = 1$ and the number of samples is $n = 1$.

## Prediction function
**Exercise**: Define a function `predict` that takes as parameter *the feature vector* $x$ and *the model* $\theta$ and returns the predicted label:
$$ \hat{y} = h(x) = \theta^T x = \sum_{j = 0}^d \theta_j x_j$$

In [2]:
def predict(x, theta):
    y_hat = x[0] * theta[0] + x[1] * theta[1]
    return y_hat

## Defining the cost function
### Cost function on a single sample
**Exercise**: Define a function `cost_function` that takes as parameter *the predicted label* $y$ and *the actual label* $\hat{y}$ of a single sample and returns the loss of this pair given by:
$$ \ell \left( y - \hat{y} \right) = \dfrac{1}{2}\left( y - \hat{y} \right)^2$$

In [3]:
def cost_function(y, y_hat):
    loss = (y - y_hat) ** 2 / 2
    return loss

### Cost function on the whole training set
Now we are able to compute the cost function for a single sample, we can easily compute the cost function for the whole training set by summing the cost function values for all the samples in the training set. Recall that the total cost function is given by:
$$J(\theta) = \dfrac{1}{2} \sum_{i = 1}^{n} \left( h\left(x^{(i)}\right) - y^{(i)} \right)^2$$
where, for all $i \in \{ 1, \dots, n \}$, we have:
$$h\left(x^{(i)}\right) = \sum_{j = 0}^{d} \theta_j x^{(i)}_j = \theta^T x.$$

In [4]:
def cost_function_total(X, Y, theta):
    cost = 0 # initialize the cost with 0
    n = len(Y)
    for i in range(n):
        x = X[i] # get the ith feature vector
        y = Y[i] # get the ith label
        y_hat = predict(x, theta) # predict the ith label
        cost += cost_function(y, y_hat) # add the cost for the current sample to the total cost
    return cost

Let's now test the code written above and check the total cost function we would have when $\theta = [0, 0]$

In [5]:
theta_0 = [0, 0]
cost_function_total(X, Y, theta_0)

5724.0

## Defining the gradient of the cost function
### Gradient on a single sample
**Exercise**: Define a function `gradient` that implements the gradient of the cost function for a given sample $(x, y)$. Recall from the lectures 1 and 2 that the gradient is given by:
$$\nabla J(\theta) = \left[ \dfrac{\partial}{\partial \theta_1} J(\theta), \dots, \dfrac{\partial}{\partial \theta_d} J(\theta) \right]^T$$
where, for all $j \in \{0, \dots, d \}$:
$$ \dfrac{\partial}{\partial \theta_j} J(\theta) = \left( h\left(x\right) - y \right) x_j $$

In [6]:
def gradient(x, y, theta):
    grad = [0, 0]
    grad[0] = (predict(x, theta) - y) * x[0] # first value of the gradient 
    grad[1] = (predict(x, theta) - y) * x[1] # second value of the gradient
    return grad

In our application, the dimensionality of our data (with the intercept) is 2, so the gradient has 2 values (corresponding to $\theta_0$ and $\theta_1$).

In [7]:
gradient(X[0], Y[0], theta_0)

[-1500.0, -30.0]

### Gradient on the whole training set
Now we are able to compute the gradient of the cost function on a single sample, we can easily compute `gradient_total`, the gradient of the cost function on the whole training set by summing the gradients for all the samples in the training set.

In [8]:
def gradient_total(X, Y, theta):
    grad_total = [0, 0] # initialize the gradient with zeros
    n = len(Y)
    for i in range(n):
        x = X[i] # get the ith feature vector
        y = Y[i] # get the ith label
        grad = gradient(x, y, theta) # predict the ith label
        grad_total[0] += grad[0] # add the gradient corresponding to theta[0]
        grad_total[1] += grad[1] # add the gradient corresponding to theta[1]
    return grad_total

Let's now test the code written above and check the total gradient we would have when $\theta = [0, 0]$

In [9]:
gradient_total(X, Y, theta_0)

[-14640.0, -180.0]

**Question**: What is the sign of the gradient values? What would it mean if we had such a gradient when applying a gradient descent?

**Hint**: Recall the gradient descent update.

## Applying a gradient descent
### Gradient descent step implementation
We now have all the building blocs needed for the gradient descent algorithm, that is:
- The loss function
- The gradient
Indeed, the iterative update scheme of this algorithm is given by the following formula:
$$\theta_j := \theta_j - \alpha \dfrac{\partial}{\partial \theta_j} J(\theta)$$
for all $j \in \{0, \dots, d \}$. Recall that $\alpha$ is a parameter called the *learning rate* (or *step size*).

**Exercise**: Define a function called `gradient_descent_step` that performs an update on theta by applying the formula above.

In [10]:
def gradient_descent_step(X, Y, theta, alpha):
    theta_updated = [0, 0]
    grad = gradient_total(X, Y, theta)
    theta_updated[0] = theta[0] - alpha * grad[0]
    theta_updated[1] = theta[1] - alpha * grad[1]
    return theta_updated

Try to run a few iterations manually. Play with the value of $\alpha$ to see how it impacts the algorithm.

In [11]:
alpha = 0.0001
theta_1 = gradient_descent_step(X, Y, theta_0, alpha)
theta_2 = gradient_descent_step(X, Y, theta_1, alpha)
theta_2

[0.0938243999999997, -0.0011927999999999973]

### Iterating gradient descent steps
The `gradient_descent_step` implements a single gradient step of the gradient descent algorithm. Implement a function called `gradient_descent` that starts from a given $\theta$ (exaple $\theta = [0, 0]$) and applies 100 gradient descent iterations. Display the total cost function $J(\theta)$ at each iteration.

In [12]:
def gradient_descent(X, Y, alpha):
    theta = [0, 0] # initializing theta with zeros (it can be initialized in another manner)
    n_iteration_max = 100
    for i_iteration in range(n_iteration_max):
        loss = cost_function_total(X, Y, theta)
        print("Iteration {:>2}. Current loss = {}".format(i_iteration, loss))
        theta = gradient_descent_step(X, Y, theta, alpha)
    loss = cost_function_total(X, Y, theta)
    print("Optimization complete. Final loss = {}".format(loss))
    return theta

Play with the code you've just run. Try different values of $\alpha$ and see the impact it has.

In [13]:
theta_trained = gradient_descent(X, Y, alpha)

Iteration  0. Current loss = 5724.0
Iteration  1. Current loss = 5037.312744000001
Iteration  2. Current loss = 4435.792673300704
Iteration  3. Current loss = 3908.876603267039
Iteration  4. Current loss = 3447.311488513006
Iteration  5. Current loss = 3042.9919278860875
Iteration  6. Current loss = 2688.8178235154955
Iteration  7. Current loss = 2378.569694239707
Iteration  8. Current loss = 2106.799453761523
Iteration  9. Current loss = 1868.7347354588892
Iteration 10. Current loss = 1660.1950836758394
Iteration 11. Current loss = 1477.5185397081189
Iteration 12. Current loss = 1317.497333241855
Iteration 13. Current loss = 1177.3215499067294
Iteration 14. Current loss = 1054.5297856756622
Iteration 15. Current loss = 946.9659215407085
Iteration 16. Current loss = 852.7412593745793
Iteration 17. Current loss = 770.2013540363196
Iteration 18. Current loss = 697.8969592515267
Iteration 19. Current loss = 634.5585770405551
Iteration 20. Current loss = 579.0741637509263
Iteration 21. Cur

**Question**: Looks at the evolution of the cost function over time. What comment can you make?

**Question**: What does the value `theta_trained` represent?

## Homework: Batch gradient descent vs. stochastic gradient descent
As we have seen during the lecture 1, the gradient descent methods are often split into 2 different subfamilies:
- **Batch methods** update $\theta$ after having computed the gradient on the whole training set
- **Stochastic methods** update $\theta$ after having computed the gradient on a single sample
The gradient descent we have implemented above (`gradient_descent_step` and `gradient_descent`) corresponds to the batch version because it sums the gradient of all the samples in the training set. 

During the next session, we will work on the following:
- Stochastic gradient descent implementation and see how it compares to the batch version of gradient descent
- Visualizing the loss function, its evolution over iterations as well as the steps that take the gradient descent algorithm.

**Exercise**: Try to implement the stochastic version of the gradient descent algorithm.